In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


import warnings
warnings.filterwarnings("ignore")

import wrangle as w
import evaluate as ev

In [2]:
zillow_outliers = w.wrangle_zillow(remove=False)

In [3]:
zillow_outliers= zillow_outliers.drop(columns="taxamount")

In [4]:
zillow_outliers.head()

,bedrooms,bathrooms,square_feet,tax_value,year_built,fips,large_home,small_home,county,county_Orange,county_Ventura,2017_age
4,4,2,3633,296425.0,2005,6037.0,False,False,LA,0,0,12
6,3,4,1620,847770.0,2011,6037.0,False,False,LA,0,0,6
7,3,2,2077,646760.0,1926,6037.0,False,False,LA,0,0,91
11,0,0,1200,5328.0,1972,6037.0,False,True,LA,0,0,45
14,0,0,171,6920.0,1973,6037.0,False,True,LA,0,0,44


In [5]:
zillow_outliers.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2140235 entries, 4 to 2152862
Data columns (total 12 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   bedrooms        2140235 non-null  int64  
 1   bathrooms       2140235 non-null  int64  
 2   square_feet     2140235 non-null  int64  
 3   tax_value       2140235 non-null  float64
 4   year_built      2140235 non-null  int64  
 5   fips            2140235 non-null  float64
 6   large_home      2140235 non-null  bool   
 7   small_home      2140235 non-null  bool   
 8   county          2140235 non-null  object 
 9   county_Orange   2140235 non-null  uint8  
 10  county_Ventura  2140235 non-null  uint8  
 11  2017_age        2140235 non-null  int64  
dtypes: bool(2), float64(2), int64(5), object(1), uint8(2)
memory usage: 155.1+ MB


In [6]:
zillow_outliers.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
bedrooms,2140235.0,NaN,NaN,NaN,3.301419,0.9326,0.0,3.0,3.0,4.0,25.0
bathrooms,2140235.0,NaN,NaN,NaN,2.186612,0.973504,0.0,2.0,2.0,3.0,32.0
square_feet,2140235.0,NaN,NaN,NaN,1863.19397,1221.754161,1.0,1258.0,1623.0,2208.0,952576.0
tax_value,2140235.0,NaN,NaN,NaN,460641.625164,677157.635675,22.0,189166.0,328296.0,534606.0,90188462.0
year_built,2140235.0,NaN,NaN,NaN,1960.967545,22.150563,1801.0,1949.0,1958.0,1976.0,2016.0
fips,2140235.0,NaN,NaN,NaN,6048.309556,20.34491,6037.0,6037.0,6037.0,6059.0,6111.0
large_home,2140235,2,False,2019146,NaN,NaN,NaN,NaN,NaN,NaN,NaN
small_home,2140235,2,False,2112157,NaN,NaN,NaN,NaN,NaN,NaN,NaN
county,2140235,3,LA,1425207,NaN,NaN,NaN,NaN,NaN,NaN,NaN
county_Orange,2140235.0,NaN,NaN,NaN,0.257942,0.437502,0.0,0.0,0.0,1.0,1.0


In [7]:
train, validate, test = w.split_data(zillow_outliers)

In [9]:
train_scaled, validate_scaled, test_scaled = w.scale_zillow(train, validate, test, scale_features=['bedrooms','bathrooms','square_feet'])






In [10]:
train_scaled.head()

,bedrooms,bathrooms,square_feet,tax_value,year_built,fips,large_home,small_home,county,county_Orange,county_Ventura,2017_age
894766,-0.269184,-0.090452,-0.471004,424000.0,1958,6037.0,False,False,LA,0,0,59
1229589,-0.269184,-0.090452,-1.210137,381638.0,1976,6037.0,False,False,LA,0,0,41
319895,-0.269184,-0.090452,-0.276998,280030.0,1958,6059.0,False,False,Orange,1,0,59
548848,-0.269184,0.922330,0.688468,249000.0,2005,6037.0,False,False,LA,0,0,12
152577,0.726606,0.922330,0.657005,336285.0,1963,6037.0,False,False,LA,0,0,54


In [27]:
def prep_for_model(train, validate, test, target):
    '''
    Takes in train, validate, and test data frames
    then splits  for X (all variables but target variable) 
    and y (only target variable) for each data frame
    '''

    X_train = train.drop(columns=[target] + ['taxamount'])
    y_train = train[target]

    X_validate = validate.drop(columns=[target] + ['taxamount'])
    y_validate = validate[target]

    X_test = test.drop(columns=[target] + ['taxamount'])
    y_test = test[target]

    return X_train, y_train, X_validate, y_validate, X_test, y_test


In [34]:
def big_zillow_wrangle(target, new = False, remove = True):
    '''
    Takes in the target variable, if you want new data, and 
    if you want to remove outliers. 
    Returns a cleaned zillow dataframe and split dataframe ready for 
    exploration and modeling
    '''
    #acquire data
    mydf = w.aquire_zillow_data(new = new)
    #clean data
    mydf = w.clean_zillow(mydf, remove = remove)
    #split data
    train, validate, test = w.split_data(mydf)
    #scale data
    train_scaled, validate_scaled, test_scaled = w.scale_zillow(train, validate, test)
    #prep for model
    X_train, y_train, X_validate, y_validate, X_test, y_test = w.prep_for_model(train_scaled, validate_scaled, test_scaled, target)
    #explore data
    X_train_exp = train.drop(columns=target)
    
    return mydf, X_train_exp, X_train, y_train, X_validate, y_validate, X_test, y_test





## Let's try leaving in outliers

In [2]:
mydf, X_train_exp, X_train, \
y_train, X_validate, y_validate, \
X_test, y_test = w.big_zillow_wrangle('tax_value', remove=False, new = True)



In [3]:
mydf.shape

(2142803, 12)

In [4]:
X_train.shape

(1392821, 8)

In [5]:
X_train.head()

,bedrooms,bathrooms,square_feet,year_built,fips,county_Orange,county_Ventura,2017_age
1398696,-0.267884,-0.092972,0.393847,1993,6059.0,1,0,24
2136681,-1.334066,-1.244778,-1.369518,1948,6037.0,0,0,69
362839,1.644369,1.649237,1.491765,1992,6059.0,1,0,25
1109882,0.728240,-0.092972,0.775680,1969,6111.0,0,1,48
1237653,-0.267884,0.918498,0.450080,2001,6037.0,0,0,16


In [6]:
y_train

1398696     301493.0
2136681      83285.0
362839     1012552.0
1109882     855000.0
1237653     370781.0
             ...    
563822      162875.0
132098      108332.0
2120742     946614.0
1985576      98071.0
1006514     154173.0
Name: tax_value, Length: 1392821, dtype: float64

In [7]:
# We need y_train and y_validate to be dataframes to append the new columns with predicted values. 
y_train_baseline = pd.DataFrame(y_train)
y_validate_baseline = pd.DataFrame(y_validate)



In [8]:
# 1. Predict G3_pred_mean
tax_value_pred_mean = y_train_baseline['tax_value'].mean()
y_train_baseline['tax_value_pred_mean'] = tax_value_pred_mean
y_validate_baseline['tax_value_pred_mean'] = tax_value_pred_mean

# 2. compute G3_pred_median
tax_value_pred_median = y_train_baseline['tax_value'].median()
y_train_baseline['tax_value_pred_median'] = tax_value_pred_median
y_validate_baseline['tax_value_pred_median'] = tax_value_pred_median

In [9]:
# 3. RMSE of G3_pred_mean
rmse_train = mean_squared_error(y_train_baseline.tax_value, y_train_baseline.tax_value_pred_mean)**(1/2)
rmse_validate = mean_squared_error(y_validate_baseline.tax_value, y_validate_baseline.tax_value_pred_mean)**(1/2)

print(f"RMSE using Mean\nTrain/In-Sample: {round(rmse_train,2)} \nValidate/Out-of-Sample: {round(rmse_validate,2)}\n")
                                                                                           
                                                                                           

RMSE using Mean
Train/In-Sample: 676977.29 
Validate/Out-of-Sample: 687354.17



In [10]:
# 4. RMSE of G3_pred_median
rmse_train = mean_squared_error(y_train_baseline.tax_value, y_train_baseline.tax_value_pred_median)**(1/2)
rmse_validate = mean_squared_error(y_validate_baseline.tax_value, y_validate_baseline.tax_value_pred_median)**(1/2)

print(f"RMSE using Median\nTrain/In-Sample: {round(rmse_train,2)} \nValidate/Out-of-Sample: {round(rmse_validate,2)}\n")

RMSE using Median
Train/In-Sample: 689813.81 
Validate/Out-of-Sample: 700157.79



In [11]:
tax_value_pred_mean

460463.95715242665

In [17]:
predictions = pd.DataFrame({
    'actual': y_validate
}) 

In [18]:
predictions.head()

,actual
803757,218988.0
486564,280431.0
1603951,506703.0
1004029,784925.0
823379,142095.0


## Linear Regression Model

In [19]:
# generate parameters, i.e. create model
lm = LinearRegression().fit(X_train, y_train)

# compute predictions
predictions['simple_lm'] = lm.predict(X_validate)

## RFE

In [24]:
rfe = RFE(lm, n_features_to_select=3)
rfe.fit(X_train, y_train)
print('selected top 2 features:', X_train.columns[rfe.support_])
X_train_rfe = rfe.transform(X_train)

selected top 2 features: Index(['year_built', 'county_Ventura', '2017_age'], dtype='object')


In [25]:
# 2. Use the transformed x in our model
lm.fit(X_train_rfe, y_train)

# 3. Make predictions
X_validate_rfe = rfe.transform(X_validate)
predictions['multiple_rfe'] = lm.predict(X_validate_rfe)

predictions.head()

,actual,simple_lm,multiple_rfe
803757,218988.0,63744.0,414720.0
486564,280431.0,699648.0,499456.0
1603951,506703.0,815616.0,711296.0
1004029,784925.0,1263616.0,644736.0
823379,142095.0,315904.0,263296.0


In [26]:
from sklearn.preprocessing import PolynomialFeatures

In [29]:
# 1. Generate Polynomial Features
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)
poly.fit(X_train)
X_train_poly = pd.DataFrame(
    poly.transform(X_train),
    columns=poly.get_feature_names(X_train.columns),
    index=X_train.index,
)
X_train_poly.head()

,bedrooms,bathrooms,square_feet,year_built,fips,county_Orange,county_Ventura,2017_age,bedrooms^2,bedrooms bathrooms,...,fips^2,fips county_Orange,fips county_Ventura,fips 2017_age,county_Orange^2,county_Orange county_Ventura,county_Orange 2017_age,county_Ventura^2,county_Ventura 2017_age,2017_age^2
1398696,-0.267884,-0.092972,0.393847,1993.0,6059.0,1.0,0.0,24.0,0.071762,0.024906,...,36711481.0,6059.0,0.0,145416.0,1.0,0.0,24.0,0.0,0.0,576.0
2136681,-1.334066,-1.244778,-1.369518,1948.0,6037.0,0.0,0.0,69.0,1.779732,1.660617,...,36445369.0,0.0,0.0,416553.0,0.0,0.0,0.0,0.0,0.0,4761.0
362839,1.644369,1.649237,1.491765,1992.0,6059.0,1.0,0.0,25.0,2.703948,2.711953,...,36711481.0,6059.0,0.0,151475.0,1.0,0.0,25.0,0.0,0.0,625.0
1109882,0.728240,-0.092972,0.775680,1969.0,6111.0,0.0,1.0,48.0,0.530334,-0.067706,...,37344321.0,0.0,6111.0,293328.0,0.0,0.0,0.0,1.0,48.0,2304.0
1237653,-0.267884,0.918498,0.450080,2001.0,6037.0,0.0,0.0,16.0,0.071762,-0.246051,...,36445369.0,0.0,0.0,96592.0,0.0,0.0,0.0,0.0,0.0,256.0


In [30]:
# 2. Use the features
lm = LinearRegression()
lm.fit(X_train_poly, y_train)

X_validate_poly = poly.transform(X_validate)
predictions['polynomial degree 2'] = lm.predict(X_validate_poly)

In [31]:
predictions.head()

,actual,simple_lm,multiple_rfe,polynomial degree 2
803757,218988.0,63744.0,414720.0,204532.0
486564,280431.0,699648.0,499456.0,606744.0
1603951,506703.0,815616.0,711296.0,858860.0
1004029,784925.0,1263616.0,644736.0,2188512.0
823379,142095.0,315904.0,263296.0,157056.0


In [32]:
feature_names = poly.get_feature_names(X_train.columns)
pd.Series(lm.coef_, index=feature_names).sort_values()

fips^2                         -7.967545e+08
year_built 2017_age            -2.380864e+08
year_built^2                   -1.214475e+08
2017_age^2                     -1.166389e+08
fips county_Orange             -1.115165e+08
year_built county_Ventura      -3.955654e+07
year_built county_Orange       -3.563001e+07
fips                           -1.865471e+07
2017_age                       -1.611535e+07
county_Ventura                 -3.927312e+06
square_feet                    -1.701357e+06
county_Orange                  -1.374264e+06
bathrooms fips                 -5.960702e+05
square_feet county_Ventura     -4.600588e+05
county_Orange 2017_age         -3.587132e+05
bedrooms fips                  -2.480864e+05
square_feet county_Orange      -1.730294e+05
bedrooms square_feet           -6.911936e+04
bedrooms bathrooms             -2.898298e+04
county_Orange^2                -1.949320e+04
square_feet year_built         -1.350764e+04
square_feet 2017_age           -1.246053e+04
bedrooms^2